# Описание задачи

[Дан набор 3D примитивов](https://drive.google.com/file/d/1xM5diKjMbp_PfNu--JG2A-ZeFk9TLkks/view?usp=sharing). Необходимо решить задачу классификации объектов с помощью нейронной сети, представленных в форме облака точек.

Набор данных разбит на три части: `train`, `test`, `valid`.

Для этого нужно:

1. Используя набор данных из категории `train` обучить модель классификации.
2. Оценить качество её работы на объектах из категории `test`. Для этого воспользоваться [classification_report из scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html?highlight=report#sklearn.metrics.classification_report). Провести финальную оценку на `valid`. Написать краткий отчёт по результатам оценки качества модели: насколько хорошо/плохо работает модель, насколько она обобщается на данные из `valid` набора, что можно сделать для улучшения результатов.


Задачу можно решать в [Google Colab](https://colab.research.google.com/), если нет возможности использовать GPU.

# Предупреждение

Решение данного задания использует shell-скрипты - для воспроизведения примеров рекомендую использовать нативный Linux (например, [Ubuntu 20.04](https://ubuntu.com/download/desktop)) или [WSL](https://docs.microsoft.com/ru-ru/windows/wsl/install-win10).

# Решение

Возьмём один из [наиболее известных](https://paperswithcode.com/task/3d-object-classification) методов классификаций трёхмерных объектов - [MeshCNN](https://github.com/ranahanocka/MeshCNN) - обучим его и оценим качество на соответствующих подвыборках. 

Прежде всего, загрузим набор примитивов **(перед запуском следующей клетки убедитесь, что у вас установлен `unzip`)**:

In [1]:
!./scripts/get_data.sh

Downloading...
From: https://drive.google.com/uc?id=1xM5diKjMbp_PfNu--JG2A-ZeFk9TLkks
To: /mnt/c/Users/semyo/PycharmProjects/test-tasks-3dml/Task3/dataset-v2.zip
10.4MB [00:01, 9.39MB/s]


Запустим обучение. Большинство параметров взяты из `MeshCNN/scripts/shrec/train.sh`, исключения:
- `ninput_edges` увеличен с запасом, потому что есть примитивы с большим числом граней, чем в SHREC. Пример: тор (`torus`).
- по той же причине увеличен `pool_res`
- `niter` и `niter_decay` снижены со 100 и 500 до 15 и 10 соответственно, потому что 100% точность на `test` чаще всего (`seed` не фиксирован) достигается к 15-й эпохе
- `print_freq` (который стоило бы назвать `print_period` или `iters_per_print`) увеличен впятеро для услады глаз и колёсика мыши

*Перед запуском обучения убедитесь, что ваше устройство поддерживает CUDA и что необходимые nvidia-драйвера установлены. Если вы не собираетесь задействовать GPU, можете пропустить следующую клетку - все остальные скрипты GPU не требуют*

In [ ]:
!./scripts/train.sh

Также вместо обучения с нуля можно воспользоваться моделью с "готовыми" весами, полученными с помощью скрипта из предыдущей клетки:

In [2]:
!./scripts/get_pretrained.sh

Downloading...
From: https://drive.google.com/uc?id=1YbGrSbcanFTKFbIaH-kKDcaqXy4JdNeG
To: /mnt/c/Users/semyo/PycharmProjects/test-tasks-3dml/Task3/latest_net.pth
5.30MB [00:00, 7.81MB/s]


Оценим точность на `test` и `valid`. Скрипт вызывает `MeshCNN/sklearn_test.py`

In [6]:
!./scripts/estimate.sh test

loaded mean / std from cache
loading the model from ../MeshCNN/checkpoints/primitives/latest_net.pth
              precision    recall  f1-score   support

        cone       1.00      1.00      1.00       100
        cube       1.00      1.00      1.00       100
    cylinder       1.00      1.00      1.00       100
       plane       1.00      1.00      1.00       100
       torus       1.00      1.00      1.00       100
   uv_sphere       1.00      1.00      1.00       100

    accuracy                           1.00       600
   macro avg       1.00      1.00      1.00       600
weighted avg       1.00      1.00      1.00       600



In [8]:
!./scripts/estimate.sh valid

loaded mean / std from cache
loading the model from ../MeshCNN/checkpoints/primitives/latest_net.pth
              precision    recall  f1-score   support

        cone       1.00      1.00      1.00        50
        cube       1.00      1.00      1.00        50
    cylinder       1.00      1.00      1.00        50
       plane       1.00      1.00      1.00        50
       torus       1.00      1.00      1.00        50
   uv_sphere       1.00      1.00      1.00        50

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300



Наблюдается успешное обобщение модели на `valid`. Так как наилучший в смысле точности результат уже достигнут, дальнейшие улучшения модели могут включать себя лишь меры по повышению производительности: например, эксперименты с уменьшением числа слоёв или фильтров свёртки в отдельных слоях.